In [7]:
%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from pinatrace_dataset import read_trace as pin_read_trace

trace = pin_read_trace()

pcs = list(set(t[0] for t in trace))
addrs = list(set(t[2] for t in trace))

print(f"Number of pcs: {len(pcs)}")
print(f"Number of addrs: {len(addrs)}")

Number of pcs: 81
Number of addrs: 39


In [40]:
from rnn import PCRnn, train_model
from collections import deque
from mapping import Mapping
import torch
import tqdm


pc_mapping = Mapping(max_size=100)

train_config = dict(
    wd=1e-4,
    history_size=8,
    pc_mapping=pc_mapping,
    sequence_rw=trace,
    train_samples=100,
)
model_config = dict(
    pc_emb_size=4,
    hidden_size=64,
    dropout=0.1,
    pc_mapping=pc_mapping,
    rnn_layers=2,
)
model = PCRnn(**model_config)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)


samples = []
current = deque(maxlen=train_config['history_size'])
bar = tqdm.tqdm(enumerate(trace), total=len(trace))
previous_pc = None
loss = float('inf')
for t, (pc, _, _) in bar:
    pc = pc_mapping(pc)
    if len(current) == train_config['history_size']:
        samples.append({'pc_history': list(current), 'next_pc': pc})

    if len(samples) == train_config['train_samples']:
        #print(samples[-1])
        loss = train_model(model, optimizer, samples)
        bar.set_postfix(loss=loss)
        samples = []

    current.append(pc)

100%|██████████| 260078/260078 [01:37<00:00, 2656.98it/s, loss=4.45]
